# Qwen Voice Clone POC
## 音声クローニング概念実証

このノートブックは、Qwen ASR + TTS を使用した音声クローニングのワークフローを実装します。

### ワークフロー:
1. **Qwen ASR Loader** - 7秒の元音声を読み込み、文字起こし＋話者特徴を抽出
2. **音声プレビュー** - 元の声をそのまま確認
3. **テキスト入力** - 読ませたい文章を入力
4. **Qwen TTS/Voice Clone** - ASRで取った「話し方」を条件に新しいテキストを音声生成
5. **オーディオ出力** - 30秒くらいの生成音声を再生・ダウンロード

## 1. 環境セットアップ

In [1]:
# 必要なライブラリのインストール
!pip install -q transformers torch torchaudio librosa soundfile ipywidgets

import torch
import torchaudio
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio, display
import ipywidgets as widgets
from google.colab import files
import io

print("✓ ライブラリのインポート完了")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.9 MB/s eta 0:00:00
✓ ライブラリのインポート完了
✓ PyTorch version: 2.9.0+cu126
✓ CUDA available: True


## 2. Qwen ASR Loader - 音声読み込みと特徴抽出

In [2]:
# 音声ファイルのアップロード
print("7秒程度の音声ファイルをアップロードしてください")
uploaded = files.upload()

# アップロードされたファイルを取得
audio_filename = list(uploaded.keys())[0]
print(f"\n✓ アップロード完了: {audio_filename}")

7秒程度の音声ファイルをアップロードしてください


Saving 無題.wav to 無題.wav

✓ アップロード完了: 無題.wav


In [3]:
# 音声の読み込みと前処理
def load_audio(filepath, target_sr=16000, target_duration=7.0):
    """
    音声ファイルを読み込み、リサンプリング
    長い音声の場合は中央部分から指定秒数を切り出す
    """
    # 音声を読み込み（モノラル化、リサンプリング）
    audio, sr = librosa.load(filepath, sr=target_sr, mono=True)
    duration = len(audio) / sr
    
    print(f"✓ 音声読み込み完了")
    print(f"  - 元のサンプリングレート: {sr} Hz")
    print(f"  - 元の長さ: {duration:.2f} 秒")
    print(f"  - 元のサンプル数: {len(audio):,}")
    
    # 音声が長すぎる場合は中央部分を切り出す
    if duration > target_duration:
        print(f"\n⚠️ 音声が{target_duration}秒より長いため、中央部分から{target_duration}秒を切り出します")
        
        # 中央部分の開始位置を計算
        center_sample = len(audio) // 2
        target_samples = int(target_duration * sr)
        half_target = target_samples // 2
        
        # 切り出し範囲を計算
        start_sample = max(0, center_sample - half_target)
        end_sample = min(len(audio), start_sample + target_samples)
        
        # 音声を切り出し
        audio = audio[start_sample:end_sample]
        duration = len(audio) / sr
        
        print(f"✓ 切り出し完了")
        print(f"  - 切り出し位置: {start_sample/sr:.2f}秒 ～ {end_sample/sr:.2f}秒")
        print(f"  - 新しい長さ: {duration:.2f} 秒")
        print(f"  - 新しいサンプル数: {len(audio):,}")
    
    elif duration < 3.0:
        print(f"\n⚠️ 警告: 音声が短すぎます（{duration:.2f}秒）")
        print(f"  推奨: 3秒以上の音声を使用してください")
    
    else:
        print(f"\n✓ 音声の長さは適切です（{duration:.2f}秒）")
    
    return audio, sr

# 音声を読み込む（7秒を目標に、長ければ中央から切り出し）
source_audio, sample_rate = load_audio(audio_filename, target_sr=16000, target_duration=7.0)

✓ 音声読み込み完了
  - サンプリングレート: 16000 Hz
  - 長さ: 9.19 秒
  - サンプル数: 146963


In [13]:
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import numpy as np

# GPUが利用可能であればGPUを使用、そうでなければCPUを使用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ 使用デバイス: {device}")

# Qwen-Audio-Speech モデルとプロセッサのロード
qwen_model_name = "Qwen/Qwen-Audio-Speech"
# Hugging Faceトークンを使用してモデルをロードする（認証エラー対策）
processor = AutoProcessor.from_pretrained(qwen_model_name, token=True)
qwen_model = AutoModelForSpeechSeq2Seq.from_pretrained(qwen_model_name, token=True).to(device)

print(f"✓ Qwen-Audio-Speech モデルとプロセッサをロードしました: {qwen_model_name}")

# ASR: 文字起こし
def transcribe_audio_qwen(audio, sr):
    """
    音声を文字起こし (Qwen-Audio-Speech ASR モデルを使用)
    """
    print("🎙️ 文字起こし中...")
    inputs = processor(audio=audio, sampling_rate=sr, return_tensors="pt").to(device)
    input_features = inputs.input_features

    # トークンIDを生成
    predicted_ids = qwen_model.generate(input_features)
    # テキストにデコード
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    print("✓ 文字起こし完了")
    print(f"  転写テキスト: {transcription}")
    return transcription

# 話者参照音声の準備（Qwen-Audio-Speech は参照音声を直接TTSに使用）
def prepare_speaker_reference(audio, sr):
    """
    Qwen-Audio-Speech TTSが参照音声として使用する準備をする。
    多くの場合、Qwen-Audio-Speechは話者クローニングのために生のオーディオを直接受け取ります。
    """
    print("🗣️ 話者参照音声を準備中...")
    # Qwen-Audio-Speech TTS は `audio` 引数で参照オーディオを受け取るため、
    # ここでは元のオーディオをそのまま返します。
    print("✓ 話者参照音声の準備完了")
    return audio

# 実行
transcription = transcribe_audio_qwen(source_audio, sample_rate)
speaker_reference_audio = prepare_speaker_reference(source_audio, sample_rate)

✓ 使用デバイス: cuda


OSError: Qwen/Qwen-Audio-Speech is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

## 3. 音声プレビュー - 元の声を確認

In [5]:
# 元音声の再生
print("🔊 元音声を再生:")
display(Audio(source_audio, rate=sample_rate))

🔊 元音声を再生:


## 4. テキスト入力 - 読ませたい文章

In [6]:
# テキスト入力ウィジェット
text_input = widgets.Textarea(
    value='吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。',
    placeholder='ここに読ませたい文章を書いてください',
    description='入力テキスト:',
    layout=widgets.Layout(width='100%', height='120px')
)

display(text_input)
print("\n📝 上のテキストボックスに読ませたい文章を入力してください")

Textarea(value='吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。', description='入力テキスト:', layout=Layout(height='120px', width…


📝 上のテキストボックスに読ませたい文章を入力してください


## 5. Qwen TTS / Voice Clone - 音声生成

In [14]:
import torch
# processor and qwen_model are already loaded from cell 0xhSllyUtaFB
import soundfile as sf
import numpy as np
# torch.nn as nn is no longer needed

print(f"✓ Qwen-Audio-SpeechモデルをTTSに使用します。")

def synthesize_voice_qwen(text, reference_audio, sr):
    """
    Qwen-Audio-Speech モデルで話者参照音声を使って音声を生成
    """
    print("🎙️ 音声生成中...")
    print(f"  入力テキスト: {text[:50]}...")
    print(f"  話者参照音声: shape={reference_audio.shape}, sr={sr} Hz")

    # テキストを処理し、input_ids を取得
    input_ids = processor(text=text, return_tensors="pt").input_ids.to(device)

    # Qwen-Audio-Speech の generate_speech メソッドで音声を生成
    # reference_audio を `audio` 引数として渡します。
    speech = qwen_model.generate_speech(input_ids, audio=reference_audio, sampling_rate=sr)

    print("✓ 音声生成完了")
    # 生成された音声の長さを計算
    duration = len(speech) / sr
    print(f"  生成音声長: {duration:.2f} 秒")

    return speech.cpu().numpy(), sr

# 音声を生成
input_text = text_input.value
generated_audio, gen_sr = synthesize_voice_qwen(input_text, speaker_reference_audio, sample_rate)

✓ Qwen-Audio-SpeechモデルをTTSに使用します。


NameError: name 'speaker_reference_audio' is not defined

## 6. オーディオ出力 - 生成音声の再生とダウンロード

In [8]:
# 生成音声の再生
print("🔊 生成音声を再生:")
display(Audio(generated_audio, rate=gen_sr))

🔊 生成音声を再生:


In [9]:
# 音声ファイルとして保存・ダウンロード
output_filename = "generated_voice_clone.wav"

# WAVファイルとして保存
sf.write(output_filename, generated_audio, gen_sr)
print(f"✓ 音声ファイル保存完了: {output_filename}")

# ダウンロード
print("\n📥 ファイルをダウンロード中...")
files.download(output_filename)
print("✓ ダウンロード完了")

✓ 音声ファイル保存完了: generated_voice_clone.wav

📥 ファイルをダウンロード中...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ ダウンロード完了


## 7. まとめと次のステップ

### 実装済み:
- ✓ 音声アップロードと読み込み
- ✓ 文字起こし（ダミー）
- ✓ 話者特徴抽出（ダミー）
- ✓ 音声プレビュー
- ✓ テキスト入力UI
- ✓ 音声生成（ダミー）
- ✓ 生成音声の再生とダウンロード

### 実装が必要な項目:
1. **Qwen ASR モデルの統合**
   - `transformers` ライブラリから Qwen ASR をロード
   - 実際の文字起こし処理
   - 話者埋め込みの抽出

2. **Qwen TTS モデルの統合**
   - Qwen TTS モデルのロード
   - 話者条件付き音声生成
   - 品質向上のための後処理

3. **パフォーマンス最適化**
   - GPU アクセラレーション
   - バッチ処理
   - メモリ管理

4. **エラーハンドリング**
   - 音声ファイル形式の検証
   - 長さの制限チェック
   - モデル推論エラーの処理